# 1. Tracking simple objects in videos

In [70]:
import cv2  
import numpy as np  
import matplotlib.pyplot as plt

In [71]:
video_path = 'video sequences/synthetic/escrime-4-3.avi'  # Chemin vers la vidéo
N = 500  # Nombre de particules
sigma = np.diag([40, 40])  # Bruit gaussien pour la prédiction (modèle de transition)
lambda_likelihood = 1  # Paramètre \lambda pour le calcul de la vraisemblance
hist_bins = 256  # Nombre de bins pour l'histogramme de couleurs

In [72]:
cap = cv2.VideoCapture(video_path)

In [73]:
ret, frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la vidéo")
else:
    # Sélectionner la zone à suivre manuellement
    roi = cv2.selectROI(frame)
    x, y, w, h = roi
    cv2.destroyWindow("ROI Selector")

In [74]:
def calc_histogram(image, roi):
    x, y, w, h = roi
    roi_image = image[int(y):int(y+h), int(x):int(x+w)]
    hist = cv2.calcHist([roi_image], [0], None, [hist_bins], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Calculer l'histogramme de la région initiale
roi_hist = calc_histogram(frame, roi)

print(w,h,x,y)

36 34 303 224


In [75]:
particles = np.column_stack((
    np.random.uniform(x, x + w, N),
    np.random.uniform(y, y + h, N)
))
weights = np.ones(N) / N  # Poids initiaux uniformes
print(particles)

[[324.62522702 236.71559893]
 [335.43439306 253.61480348]
 [324.0940106  235.64958829]
 [315.29712944 245.65585309]
 [330.69098929 249.38290662]
 [309.07122308 231.41234104]
 [331.61846793 242.19952234]
 [318.63139345 250.24851105]
 [324.04047029 241.50075872]
 [328.35675168 250.81869335]
 [336.93129395 255.23023947]
 [325.99125977 239.01157561]
 [311.23613179 236.8396741 ]
 [318.20091073 240.19759116]
 [303.95324316 251.6122783 ]
 [307.08783624 233.83049665]
 [320.92634996 253.99224265]
 [326.89684648 239.21305651]
 [327.16637821 225.96178208]
 [338.47734455 254.8599752 ]
 [319.8552455  231.80008817]
 [316.63970782 237.86423842]
 [317.32012714 244.82319316]
 [329.63189522 232.89139819]
 [326.91378587 249.60292509]
 [334.09611758 234.78037881]
 [328.86337891 242.04800715]
 [322.54718333 225.46920712]
 [328.3449036  252.6120902 ]
 [332.63004754 254.24785421]
 [306.67861453 230.1544005 ]
 [321.7306104  256.01608352]
 [313.79481044 257.80720813]
 [321.87951542 225.51444089]
 [335.23309052

In [76]:
def predict(particles, sigma):
    noise = np.random.multivariate_normal([0, 0], sigma, N)
    particles += noise
    return particles

In [77]:
def compute_weights(particles, frame, roi_hist, lambd=lambda_likelihood):
    weights = np.zeros(N)
    for i, particle in enumerate(particles):
        px, py = particle
        # Définir une petite zone autour de chaque particule
        patch_hist = calc_histogram(frame, (px, py, w, h))

        # Calculer la distance entre les deux histogrammes 
        dist = cv2.compareHist(roi_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)

        # Mettre à jour les poids selon la fonction de vraisemblance
        weights[i] = np.exp(-lambd * dist**2)

    # Normaliser les poids pour qu'ils forment une distribution de probabilité
    weights /= np.sum(weights)
    return weights

In [78]:
def resample(particles, weights):
    """
    Rééchantillonne les particules en fonction de leurs poids.
    
    Parameters:
    particles (numpy.ndarray): Les particules (positions) actuelles, de taille (N, 2).
    weights (numpy.ndarray): Les poids actuels des particules, de taille (N,).
    
    Returns:
    numpy.ndarray: Les nouvelles particules rééchantillonnées, de taille (N, 2).
    """
    N = len(particles)
    indices = np.arange(N)

    # Choisir des particules selon leurs poids en utilisant la méthode du rééchantillonnage systématique
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1.0  # Pour s'assurer que la somme est exactement 1
    u0 = np.random.uniform(0, 1/N)
    positions = (u0 + np.arange(N) / N)  # Échantillons également espacés

    # Indices des particules sélectionnées
    new_indices = np.searchsorted(cumulative_sum, positions)

    # Créer un nouvel ensemble de particules rééchantillonnées
    resampled_particles = particles[new_indices]

    return resampled_particles

What if we skip on frame for every two consecutive frames ?

In [79]:
frame_skip = 1  # Sauter une frame sur deux

In [80]:
# Boucle principale avec frame_skip pour ignorer des frames
while True:
    for _ in range(frame_skip):  # Sauter des frames si nécessaire
        ret, frame = cap.read()
        if not ret:
            break 

    # Prédiction : mise à jour des particules avec du bruit
    particles = predict(particles, sigma)

    # Correction : calcul des poids à partir des histogrammes
    weights = compute_weights(particles, frame, roi_hist)

    # Rééchantillonnage : sélection des particules en fonction des poids
    particles = resample(particles, weights)

    # Calculer la position moyenne de l'objet après rééchantillonnage
    estimated_position = np.average(particles, axis=0, weights=weights)

    # Dessiner le rectangle autour de la position estimée
    #cv2.rectangle(frame,
                  #(int(estimated_position[0]), int(estimated_position[1])),
                  #(int(estimated_position[0] + w), int(estimated_position[1] + h)),
                  #(0, 255, 0), 2)

    # Afficher chaque particule sur la vidéo avec un cercle bleu
    for particle in particles:
        cv2.circle(frame, (int(particle[0]), int(particle[1])), 2, (0, 255, 255), -1)  

    # Afficher l'image avec le suivi
    cv2.imshow("Tracking", frame)

    # Quitter en appuyant sur 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable